# Demo - Hallucination Detection Intrinsic

This notebook demonstrates some examples of using the [Granite hallucination detection intrinsic](https://huggingface.co/generative-computing/rag-intrinsics-lib/blob/main/hallucination_detection/README.md). It uses the shared IO processing code for intrinsics when performing model inference with an OpenAI-compatible backend such as vLLM.

In [ ]:
# Imports go in this cell
import openai

from IPython.display import display, Markdown

import granite_common
from granite_common import ChatCompletion
from granite_common.base.types import AssistantMessage
from granite_common.intrinsics.constants import BASE_MODEL_TO_CANONICAL_NAME

## Constants

In [ ]:
intrinsic_name = "hallucination_detection"
base_model_name = "ibm-granite/granite-3.3-8b-instruct"

# Change the following two constants as needed to reflect the location of the
# inference server.
openai_base_url = "http://localhost:55555/v1"
openai_api_key = "rag_intrinsics_1234"

## Instantiate IO processing classes

In [ ]:
# Fetch IO configuration file from Hugging Face Hub
io_yaml_file = granite_common.intrinsics.util.obtain_io_yaml(
    intrinsic_name, BASE_MODEL_TO_CANONICAL_NAME[base_model_name]
)

print(
    f"Instantiating input and output processing from configuration file:\n"
    f"{io_yaml_file}"
)

intrinsics_rewriter = granite_common.IntrinsicsRewriter(config_file=io_yaml_file)
intrinsincs_result_processor = granite_common.IntrinsicsResultProcessor(
    config_file=io_yaml_file
)

## Perform input processing

In [ ]:
# Create an example chat completion with a user question and two documents.
chat_input = ChatCompletion.model_validate(
    {
        "messages": [
            {
                "role": "user",
                "content": "What is the visibility level of Git Repos and Issue \
Tracking projects?",
            }
        ],
        "extra_body": {
            "documents": [
                {
                    "doc_id": "1",
                    "text": "Git Repos and Issue Tracking is an IBM-hosted \
component of \
the Continuous Delivery service. All of the data that you provide to Git Repos and \
Issue Tracking, including but not limited to source files, issues, pull requests, and \
project configuration properties, is managed securely within Continuous Delivery. \
However, Git Repos and Issue Tracking supports various mechanisms for exporting, \
sending, or otherwise sharing data to users and third parties. The ability of Git \
Repos and Issue Tracking to share information is typical of many social coding \
platforms. However, such sharing might conflict with regulatory controls that \
apply to your business. After you create a project in Git Repos and Issue Tracking, \
but before you entrust any files, issues, records, or other data with the project, \
review the project settings and change any settings that you deem necessary to \
protect your data. Settings to review include visibility levels, email notifications, \
integrations, web hooks, access tokens, deploy tokens, and deploy keys. Project \
visibility levels \n\nGit Repos and Issue Tracking projects can have one of the \
following visibility levels: private, internal, or public. * Private projects are \
visible only to project members. This setting is the default visibility level for new \
projects, and is the most secure visibility level for your data. * Internal projects \
are visible to all users that are logged in to IBM Cloud. * Public projects are \
visible to anyone. To limit project access to only project members, complete the \
following steps:\n\n\n\n1. From the project sidebar, click Settings > General. \
2. On the General Settings page, click Visibility > project features > permissions. \
3. Locate the Project visibility setting. 4. Select Private, if it is not already \
selected. 5. Click Save changes. Project membership \n\nGit Repos and Issue Tracking \
is a cloud hosted social coding environment that is available to all Continuous \
Delivery users. If you are a Git Repos and Issue Tracking project Maintainer or Owner, \
you can invite any user and group members to the project. IBM Cloud places no \
restrictions on who you can invite to a project.",
                },
                {
                    "doc_id": "2",
                    "text": "After you create a project in Git Repos and Issue \
Tracking, but before you entrust any files, issues, records, or other data with \
the project, review the project settings and change any settings that are \
necessary to protect your data. \
Settings to review include visibility levels, email notifications, integrations, web \
hooks, access tokens, deploy tokens, and deploy keys. Project visibility levels \
\n\nGit Repos and Issue Tracking projects can have one of the following visibility \
levels: private, internal, or public. * Private projects are visible only to \
project members. This setting is the default visibility level for new projects, and \
is the most secure visibility level for your data. * Internal projects are visible to \
all users that are logged in to IBM Cloud. * Public projects are visible to anyone. \
To limit project access to only project members, complete the following \
steps:\n\n\n\n1. From the project sidebar, click Settings > General. 2. On the \
General Settings page, click Visibility > project features > permissions. 3. Locate \
the Project visibility setting. 4. Select Private, if it is not already selected. \
5. Click Save changes. Project email settings \n\nBy default, Git Repos and Issue \
Tracking notifies project members by way of email about project activities. These \
emails typically include customer-owned data that was provided to Git Repos and Issue \
Tracking by users. For example, if a user posts a comment to an issue, Git Repos and \
Issue Tracking sends an email to all subscribers. The email includes information such \
as a copy of the comment, the user who posted it, and when the comment was posted. \
To turn off all email notifications for your project, complete the following \
steps:\n\n\n\n1. From the project sidebar, click Settings > General. 2. On the \
**General Settings **page, click Visibility > project features > permissions. \
3. Select the Disable email notifications checkbox. 4. Click Save changes. Project \
integrations and webhooks",
                },
            ],
        },
        "model": base_model_name,
        "temperature": 0.0,
    }
)
print(chat_input.model_dump_json(indent=2))

## Run inference

In [ ]:
# Connect to the inference server
client = openai.OpenAI(base_url=openai_base_url, api_key=openai_api_key)

In [ ]:
# Pass the example through Granite to get an answer
chat_input.model = base_model_name
chat_completion = client.chat.completions.create(**chat_input.model_dump())

chat_completion_message = chat_completion.choices[0].message

display(Markdown(chat_completion_message.content))

In [ ]:
next_chat_input = chat_input.model_copy(deep=True)
next_chat_input.messages.append(chat_completion_message)
next_chat_input.messages

In [ ]:
# Run request through intrinsics input processing
intrinsics_input = next_chat_input.model_copy(deep=True)
intrinsics_input.model = intrinsic_name

intrinsics_request = intrinsics_rewriter.transform(intrinsics_input)
print(intrinsics_request.model_dump_json(indent=2))

In [ ]:
# Get the raw intrinsics output
intrinsics_completion = client.chat.completions.create(
    **intrinsics_request.model_dump()
)
intrinsics_completion_message = intrinsics_completion.choices[0].message

print(intrinsics_completion_message.content)

## Post-process inference results

In [ ]:
processed_chat_completion = intrinsincs_result_processor.transform(
    intrinsics_completion, intrinsics_request
)

print(processed_chat_completion.choices[0].message.content)

## Poor-quality assistant response

In [ ]:
# Try with an artifical poor-quality assistant response.
alt_chat_input = chat_input.model_copy(deep=True)
alt_chat_input.messages.append(
    AssistantMessage(
        content="Git repos are generally only visible in the infrared "
        "spectrum, due to their natural camouflage. Issue Tracking projects "
        "are much easier to see; their bright colors warn predators of the "
        "poisonous technical debt that they secrete."
    )
)

alt_intrinsics_input = alt_chat_input.model_copy(deep=True)
alt_intrinsics_input.model = intrinsic_name
alt_intrinsics_request = intrinsics_rewriter.transform(alt_intrinsics_input)

alt_intrinsics_completion = client.chat.completions.create(
    **alt_intrinsics_request.model_dump()
)
alt_intrinsics_completion_message = alt_intrinsics_completion.choices[0].message

processed_chat_completion = intrinsincs_result_processor.transform(
    alt_intrinsics_completion, alt_intrinsics_request
)

print(processed_chat_completion.choices[0].model_dump_json(indent=2))

## Multiple completions

In [ ]:
import json

chat_input_5 = chat_input.model_copy(deep=True)
chat_input_5.n = 5
chat_input_5.temperature = 0.7
chat_input_5.model = base_model_name
chat_completion_5 = client.chat.completions.create(**chat_input_5.model_dump())

for choice in chat_completion_5.choices:
    choice_chat_input = chat_input.model_copy(deep=True)
    choice_chat_input.model = intrinsic_name
    choice_chat_input.n = 1
    choice_chat_input.temperature = 0.0

    intrinsics_request = intrinsics_rewriter.transform(choice_chat_input)
    intrinsics_completion = client.chat.completions.create(
        **intrinsics_request.model_dump()
    )
    # print(intrinsics_request.model_dump_json(indent=2))
    # print(intrinsics_completion.model_dump_json(indent=2))

    processed_chat_completion = intrinsincs_result_processor.transform(
        intrinsics_completion, intrinsics_request
    )

    citations = json.loads(processed_chat_completion.choices[0].message.content)

    print(f"Assistant: {choice.message.content}")
    print(f"           ({len(citations)} hallucinations)")